<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/finetuned_object_detection_using_adapters_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuned Object Detection - using adapters

In [ ]:
!pip install -q einops flash_attn timm peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 786.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.1 MB/s eta 0:00:00


In [ ]:
import requests
import torch

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

## Start with a Light Weight Vision Model - with Fine tuning capabilities

In [ ]:
base_vision_model_id = "microsoft/Florence-2-large-ft"

Set it for Object Detection mode

In [ ]:
prompt = "<OD>"

## Blood Cell Images

### Test with the Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

blood_cell_adapter = "dwb2023/florence2-large-bccd-base-ft"

model = AutoModelForCausalLM.from_pretrained(base_vision_model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(blood_cell_adapter)
processor = AutoProcessor.from_pretrained(base_vision_model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
url = "https://huggingface.co/spaces/dwb2023/omniscience/resolve/main/examples/BloodImage_00099_jpg.rf.0a65e56401cdd71253e7bc04917c3558.jpg"

image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

{'<OD>': {'bboxes': [[504.6399841308594, 124.07999420166016, 628.1599731445312, 240.239990234375], [376.6399841308594, 189.36000061035156, 480.3199768066406, 304.0799865722656], [428.47998046875, 15.119999885559082, 540.47998046875, 117.36000061035156], [124.47999572753906, 213.36000061035156, 223.0399932861328, 330.47998046875], [543.0399780273438, 364.0799865722656, 639.6799926757812, 465.3599853515625], [377.2799987792969, 407.2799987792969, 500.1600036621094, 479.7599792480469], [139.1999969482422, 345.3599853515625, 239.0399932861328, 450.47998046875], [148.16000366210938, 157.1999969482422, 255.0399932861328, 248.39999389648438], [1.5999999046325684, 232.0800018310547, 48.31999969482422, 330.47998046875], [340.1600036621094, 157.1999969482422, 364.47998046875, 181.1999969482422], [225.59999084472656, 229.1999969482422, 376.6399841308594, 345.3599853515625], [1.5999999046325684, 1.1999999284744263, 639.0399780273438, 479.2799987792969]], 'labels': ['RBC', 'RBC', 'RBC', 'RBC', 'RBC

## Switch to the Liver Disease Image Adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

liver_disease_adapter = "dwb2023/florence2-large-liver-disease-ft"

model = AutoModelForCausalLM.from_pretrained(base_vision_model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(liver_disease_adapter)
processor = AutoProcessor.from_pretrained(base_vision_model_id, trust_remote_code=True)

adapter_config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

In [ ]:
url = "https://huggingface.co/spaces/dwb2023/omniscience/resolve/main/examples/15_242_212_25_25_jpg.rf.f6bbadf4260dd2c1f5b4ace1b09b0a1b.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

{'<OD>': {'bboxes': [[0.14949999749660492, 69.21849822998047, 69.21849822998047, 173.27049255371094]], 'labels': ['fibrosis']}}
